In [1]:
# eval on predictions
import pandas as pd
from evaluator.CodeBLEU.calc_code_bleu import get_codebleu

In [17]:
def evaluate_codebleu(pred_filename):
    pred_df = pd.read_csv(pred_filename)
    # a list of gold codes (which is just some variants of the same code, we can use every code of different styles)
    refs = [
        pred_df["labels"]
    ]
    # the prediction code
    hyp = pred_df["preds"]
    score = get_codebleu(refs, hyp, "python", '0.25,0.25,0.25,0.25')
    return score

# Uncomment Parallel Corpus - no outliers

In [18]:
# no_outlier_codet5small
evaluate_codebleu("seq2seq_results/no_outlier_codet5small/codet5_preds.csv")

ngram match: 0.7020337009365258, weighted ngram match: 0.7158332483308997, syntax_match: 0.9308169419505175, dataflow_match: 0.8688835181384277


0.8043918523390927

# Removed Class Parallel Corpus - with outliers

In [23]:
# outlier_class_codet5small
evaluate_codebleu("seq2seq_results/outlier_class_codet5small/codet5_preds.csv")

ngram match: 0.9539663181494397, weighted ngram match: 0.955850824785441, syntax_match: 0.9574235970530824, dataflow_match: 0.8714937626650319


0.9346836256632487